### 3.1 Predict Messages

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1)

# 가상의 대화 설정
messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Korean."
    ),
    AIMessage(content="안녕! 나는 챗봇이야."),
    HumanMessage(content="What is the distance between Korea and Japan? Also, what is your name?"),
]

chat.predict_messages(messages)

AIMessage(content='한국과 일본 사이의 거리는 약 900km입니다. 제 이름은 챗봇이에요.')

### 3.2 Prompt Templates

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}?",
)

prompt = template.format(country_a="Korea", country_b="Vietnam")

chat.predict(prompt)

'The distance between Korea and Vietnam is approximately 2,500 kilometers (1,550 miles) when measured in a straight line.'

In [8]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "안녕! 나는 {name}이야."),
        ("human", "What is the distance between {country_a} and {country_b}? also, what is your name?"),
    ]
)

prompt = template.format_messages(
    language="Korean",
    name="chatbot",
    country_a="Korea",
    country_b="France",
)

chat.predict_messages(prompt)

AIMessage(content='한국과 프랑스 사이의 거리는 대략 9000km입니다. 제 이름은 AI 어시스턴트입니다. 어떻게 도와드릴까요?')

### 3.3 OutputParser and LCEL(LangChain Expression Language)

In [11]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    # must define parse function
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse('hi,how,are,you')

['hi', 'how', 'are', 'you']

In [16]:
# without LCEL

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
        ("human", "{question}."),
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [17]:
# with LCEL
# make a chain of chat model, OutputParser and template.

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
        ("human", "{question}."),
    ]
)

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

### 3.4 Chaining Chains

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# streaming: LLM의 응답이 생성되는걸 볼 수 있게 해줌
# StreamingStdOutCallbackHandler: console에서 응답의 진행을 볼 수 있음
chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [22]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

In [23]:
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and easy recipe for Chicken Tikka Masala.

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tsp ground cumin
- 2 tsp paprika
- 1 tsp ground turmeric
- 1 tsp garam masala
- 1 tsp ground coriander
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, turmeric, garam masala, coriander, cayenne pepper, salt, and pepper. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°F (200°C). Thread the marinated 

AIMessageChunk(content="As a vegetarian chef, I can help you make a delicious vegetarian version of Chicken Tikka Masala by replacing the chicken with a suitable alternative. In this case, I recommend using paneer, a type of Indian cottage cheese that is commonly used in vegetarian dishes as a substitute for meat. Here's how you can prepare the paneer for this recipe:\n\nIngredients:\n- 1 lb paneer, cut into bite-sized pieces\n- 1 cup plain yogurt\n- 2 tbsp lemon juice\n- 2 tsp ground cumin\n- 2 tsp paprika\n- 1 tsp ground turmeric\n- 1 tsp garam masala\n- 1 tsp ground coriander\n- 1/2 tsp cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- 2 tbsp vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) crushed tomatoes\n- 1 cup heavy cream\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, turmeric, garam masala, coriander, cayenne peppe